In [1]:
import requests
from bs4 import BeautifulSoup
from collections import deque
import certifi
import re
import nltk
import os
from itertools import chain
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import itertools
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import math
import operator
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from urllib.request import urljoin
from bs4 import BeautifulSoup
from urllib.request import urlparse
import ssl
# Set for storing urls with same domain
depth = 1
links_intern = set()

# importan variable
input_url = "https://www.uic.edu/"
hashtable = {} 
depth = 10
# ***************

docmap = {}
stop_words = set(stopwords.words("english"))
# Set for storing urls with different domain
links_extern = set()
lem = WordNetLemmatizer()
desirednum = 3200

In [2]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

In [3]:
# tokenize query or text
def tokenize(query, qMap):
    tmp = {}
    query = re.sub(r'[^A-Za-z]+', ' ', query).lower()
    token_word = word_tokenize(query)
    for word in token_word:
        if word not in stop_words and not word.isdigit():
            word = lem.lemmatize(word)
            qMap[word] = qMap.setdefault(word, 0) + 1       
    return qMap

In [4]:
def tokenizeContents(url, query, qMap):
    query = re.sub(r'[^A-Za-z]+', ' ', query).lower()
    token_word = word_tokenize(query)
    for word in token_word:
        if word not in stop_words and not word.isdigit():
            word = lem.lemmatize(word)
            # append new dict to existing dict of that word
            if word in word_tokenized:
                tmpdict = word_tokenized[word]
                tmpdict[url] = tmpdict.setdefault(url, 0) + 1 
            #create a dict to that word with corresponding url
            else:
                tmp = {}
                tmp[url] = 1
                word_tokenized[word] = tmp
                      
    return word_tokenized

In [5]:
def returnContent(input_url):
    text = ''
    try:
        r = requests.get(input_url, verify=False)
        text = BeautifulSoup(r.text, "lxml").text
        return text
    except:
        print("An exception occurred while getting content")
        return text

In [18]:
# initialize queries
query1 = {}
query2 = {}
query3 = {}
query4 = {}
query5 = {}

query1 = tokenize("appointment vaccine", query1)
query2 = tokenize("basketball court", query2)
query3 = tokenize("register course", query3)
query4 = tokenize("job board", query4)
query5 = tokenize("train station", query5)
print(query1, query2, query3, query4, query5)

{'appointment': 1, 'vaccine': 1} {'basketball': 1, 'court': 1} {'register': 1, 'course': 1} {'job': 1, 'board': 1} {'train': 1, 'station': 1}


In [7]:
# Set for storing urls with same domain
links_intern = set()
hashtable = {}
input_url = "https://www.uic.edu/"
  
# Set for storing urls with different domain
links_extern = set()
  
  
# Method for crawling a url at next level
def level_crawler(input_url):
    temp_urls = set()
    current_url_domain = urlparse(input_url).netloc
#     print(current_url_domain)
    if ".uic.edu" not in current_url_domain:
        return
    
    
    # Creates beautiful soup object to extract html tags
    try:
        r = requests.get(input_url, verify=False)
        beautiful_soup_object = BeautifulSoup(r.content, "lxml")
    except:
        print("An exception occurred")
        return
  
    # Access all anchor tags from input 
    # url page and divide them into internal
    # and external categories
    for anchor in beautiful_soup_object.findAll("a"):
        href = anchor.attrs.get("href")
        if(href != "" or href != None):
            href = urljoin(input_url, href)
            href_parsed = urlparse(href)
            href = href_parsed.scheme
            href += "://"
            href += href_parsed.netloc
            href += href_parsed.path
            final_parsed_href = urlparse(href)
            is_valid = bool(final_parsed_href.scheme) and bool(
                final_parsed_href.netloc)
            if is_valid:
                if ".uic.edu" not in href and href not in links_extern:
#                     print("Extern - {}".format(href))
                    links_extern.add(href)
                if ".uic.edu" in href and href not in links_intern:
#                     print("Intern - {}".format(href))
                    text = returnContent(href)
                    if(text ==''):
                        continue
                    else:
                        links_intern.add(href)
                        hashtable[href] = text
#                     links_intern.add(href)
#                     hashtable[href] = BeautifulSoup(r.text, "lxml").text
                    temp_urls.add(href)
    return temp_urls
  
  
if(depth == 0):
    print("Intern - {}".format(input_url))
  
elif(depth == 1):
    level_crawler(input_url)
  
else:
    # Used a BFS approach
    queue = []
    queue.append(input_url)
    for j in range(depth):
        print("depth:", j)
        for count in range(len(queue)):
            if(count %10 == 0) :
                print("links_intern", len(links_intern))
            if(len(links_intern) > desirednum):
                print("Reach 3200")
                break
            url = queue.pop(0)
            urls = level_crawler(url)
            s = requests.session()
            s.keep_alive = False
            if(not urls):
                print("Out of domain:", urls)
                continue
            for i in urls:
                queue.append(i)


depth: 0
links_intern 0


/Users/allanswer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.uic.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/allanswer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'accc.uic.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/allanswer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'it.uic.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/allanswer/opt/anac

/Users/allanswer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'oae.uic.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/allanswer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sexualmisconduct.uic.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/allanswer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'financialaid.uic.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Us

depth: 1
links_intern 30


/Users/allanswer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'advance.uic.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/allanswer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'advance.uic.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/allanswer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'advance.uic.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/allans

...

An exception occurred while getting content


/Users/allanswer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sa.uic.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/allanswer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'dining.uic.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/allanswer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'childrenscenter.uic.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/alla

An exception occurred while getting content


/Users/allanswer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sa.uic.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/allanswer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sa.uic.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/allanswer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sa.uic.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/allanswer/opt/anacond

Reach 3200
depth: 3
links_intern 3244
Reach 3200
depth: 4
links_intern 3244
Reach 3200
depth: 5
links_intern 3244
Reach 3200
depth: 6
links_intern 3244
Reach 3200
depth: 7
links_intern 3244
Reach 3200
depth: 8
links_intern 3244
Reach 3200
depth: 9
links_intern 3244
Reach 3200


In [17]:
print("Size of links_intern:", len(links_intern))
lem = WordNetLemmatizer()

Size of links_intern: 3244


In [10]:
# tokenize each contens of each links
# map[word] = {url: cnt....}
global word_tokenized
print("Total links to be tokenize:",len(hashtable))
word_tokenized = {}
progress = 0
for i in hashtable:
     
    tokenizeContents(i, hashtable[i], word_tokenized)
    
    #print progress
    progress += 1
    if(progress %50 == 0 or progress == len(hashtable)):
        print(progress)
# print(word_tokenized)

Total links to be tokenize: 3244
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3244


In [11]:

print(len(word_tokenized))

185224


In [12]:
def getTfidf_Length(hashtable, query):
    N = len(hashtable)

    tfidf_word_in_link = {}
    length_word_in_link = {}

    for qword in query:
        qtf = query[qword]
        print(qword)
        if qword in word_tokenized:
    #         print(word_tokenized[qword])
            dft = len(word_tokenized[qword])
            idf = math.log(N / (dft))
            print("In " + str(dft) + " links")
            #calcaulate tf-idf in each links
            for link in word_tokenized[qword]:
                tf = word_tokenized[qword][link]
                tf_idf = tf * idf
                dot = qtf * tf_idf
                tfidf_word_in_link[link] = tfidf_word_in_link.setdefault(link, 0) + dot
                length_word_in_link[link] = length_word_in_link.setdefault(link, 0) + pow(tf_idf, 2)
                #print(length_word_in_link)
    return  tfidf_word_in_link,  length_word_in_link         

In [20]:
# calculate sum of tf-idf to see which is relevant
#q1{go:1, chicago:1}, tf = 1, 1, idfs are same.
#caculate cosine simiiar = (Vq*Vt)/(|Vq| * |Vt|), the length of Vq are same, so we only need the length of Vt

print("Query1----------------------")
tfidf_word_in_link1, length_word_in_link1 = getTfidf_Length(hashtable, query1)

print("Query2----------------------")
tfidf_word_in_link2, length_word_in_link2 = getTfidf_Length(hashtable, query2)

print("Query3----------------------")
tfidf_word_in_link3, length_word_in_link3 = getTfidf_Length(hashtable, query3)

print("Query4----------------------")
tfidf_word_in_link4, length_word_in_link4 = getTfidf_Length(hashtable, query4)

print("Query5----------------------")
tfidf_word_in_link5, length_word_in_link5 = getTfidf_Length(hashtable, query5)

# N = len(hashtable)

# tfidf_word_in_link = {}
# length_word_in_link = {}

# for qword in query:
#     qtf = query[qword]
#     print(qword)
#     if qword in word_tokenized:
# #         print(word_tokenized[qword])
#         dft = len(word_tokenized[qword])
#         idf = math.log(N / (dft))
#         print("In " + str(dft) + " links")
#         #calcaulate tf-idf in each links
#         for link in word_tokenized[qword]:
#             tf = word_tokenized[qword][link]
#             tf_idf = tf * idf
#             dot = qtf * tf_idf
#             tfidf_word_in_link[link] = tfidf_word_in_link.setdefault(link, 0) + dot
#             length_word_in_link[link] = length_word_in_link.setdefault(link, 0) + pow(tf_idf, 2)
#             #print(length_word_in_link)
            


Query1----------------------
appointment
In 345 links
vaccine
In 107 links
Query2----------------------
basketball
In 17 links
court
In 65 links
Query3----------------------
register
In 199 links
course
In 749 links
Query4----------------------
job
In 2650 links
board
In 2924 links
Query5----------------------
train
In 54 links
station
In 74 links


In [21]:
def getResAndRank(tfidf_word_in_link, length_word_in_link):
    cosineres = {}
    for i in length_word_in_link:
        length_word_in_link[i] = math.sqrt(length_word_in_link[i])
        cosineres[i] =  tfidf_word_in_link[i] / length_word_in_link[i]

    # Sort results
    sorted_cosine = dict( sorted(cosineres.items(), key=operator.itemgetter(1),reverse=True))
    cnt = 0
    for link in sorted_cosine:
        print("No." + str(cnt + 1) + " link: " + link + " ," + str(sorted_cosine[link]))
        cnt += 1
        if(cnt == 10): break
    

In [26]:
#Get results of cosine similarities and rank them


print("Ranks of Query1----------------------" + "appointment vaccine")
getResAndRank(tfidf_word_in_link1, length_word_in_link1)
print('\n')

print("Ranks of Query2----------------------" + "basketball court")
getResAndRank(tfidf_word_in_link2, length_word_in_link2)
print('\n')

print("Ranks of Query3----------------------" + "register course")
getResAndRank(tfidf_word_in_link3, length_word_in_link3)
print('\n')

print("Ranks of Query4----------------------" + "job board")
getResAndRank(tfidf_word_in_link4, length_word_in_link4)
print('\n')

print("Ranks of Query5----------------------" + "train station")
getResAndRank(tfidf_word_in_link5, length_word_in_link5)
print('\n')

# cosineres = {}
# for i in length_word_in_link:
#     length_word_in_link[i] = math.sqrt(length_word_in_link[i])
#     cosineres[i] =  tfidf_word_in_link[i] / length_word_in_link[i]
    
# # Sort results
# sorted_cosine = dict( sorted(cosineres.items(), key=operator.itemgetter(1),reverse=True))
# cnt = 0
# for link in sorted_cosine:
#     print("No." + str(cnt + 1) + " link: " + link + " ," + str(sorted_cosine[link]))
#     cnt += 1
#     if(cnt == 10): break

Ranks of Query1----------------------appointment vaccine
No.1 link: https://uihealth.uic.edu/covid-19-information-resources/covid-19_vaccine-info/ ,164.38713120184389
No.2 link: https://today.uic.edu/frequently-asked-questions ,149.77397490374395
No.3 link: https://wellnesscenter.uic.edu/news-stories/facts-on-covid-19-vaccine/ ,114.41939128341261
No.4 link: https://today.uic.edu/flu-shots ,86.89094592907166
No.5 link: https://studyabroad.uic.edu/students/get-started/advising/ ,51.682520640570004
No.6 link: https://ready.uic.edu/news-stories/vaccine-appointments-available-for-uic-employees-schedule-today/ ,51.52966556706401
No.7 link: https://today.uic.edu/uic-works-with-chicago-to-deliver-vaccinations-to-citys-most-vulnerable ,45.250420574501774
No.8 link: https://careerservices.uic.edu/about-us/frequently-asked-questions/ ,40.286761427052056
No.9 link: https://advance.uic.edu/news-stories/call-to-action/ ,32.46359292643714
No.10 link: https://wellnesscenter.uic.edu/news-stories/when-y

In [27]:
print("finish")

finish
